In [15]:
from PIL import Image
import pytesseract
from ultralyticsplus import YOLO
import torch

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Load the YOLO model for table detection
model = YOLO('foduucom/table-detection-and-extraction')

class FindNutrition:
    def __init__(self, image, text: list):
        self.image = image
        self.text = text

    def detect_tables(self):
        # Perform table detection
        results = model.predict(self.image)
        return results[0].boxes

    def crop_table(self, box):
        # Extract the bounding box coordinates and convert to integers
        left, top, right, bottom = map(int, box.xyxy[0])
        return self.image.crop((left, top, right, bottom))

    def ocr(self, image):
        data = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)
        result = []
        for i, word in enumerate(data['text']):
            if word.strip():  # Filter out empty results
                result.append({
                    "word": word,
                    "confidence": data['conf'][i],
                    "bounding_box": {
                        "left": data['left'][i],
                        "top": data['top'][i],
                        "width": data['width'][i],
                        "height": data['height'][i]
                    }
                })
        return result

    def plain_text_ocr(self, image):
        return pytesseract.image_to_string(image)

    def find_text_ocr(self, image):
        found_text = []
        data = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)
        for i, word in enumerate(data['text']):
            if word.strip():  # Filter out empty results
                for j in self.text:
                    if j.lower() in word.lower():
                        found_text.append(word)
        return found_text

c:\Users\phumi\anaconda3\envs\ocr\Lib\site-packages\ultralytics\nn\tasks.py:340: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), f

In [18]:
# Example usage in code:
image = Image.open('./images/n1.jpg')
nutrition_finder = FindNutrition(image, ['calories', 'fat', 'sugar'])
tables = nutrition_finder.detect_tables()
if not tables:
    image_to_use = image
    ocr_results = nutrition_finder.find_text_ocr(image_to_use)
    ocr_results = [l.lower() for l in ocr_results]
    results = set(ocr_results) 
    print(results)
else:
    for table in tables:
        cropped_table = nutrition_finder.crop_table(table)
        ocr_results = nutrition_finder.find_text_ocr(cropped_table)
        ocr_results = [l.lower() for l in ocr_results]
        results = set(ocr_results)
        print(results)


0: 640x512 (no detections), 158.6ms
Speed: 3.0ms preprocess, 158.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


{'fat', 'sugars]', 'calories', 'sugars'}
